# Project goal :
To try and understand the fate of pronouns upon backtranslation using Google translate.

## Notebook goals
- Generate the dataset entailing 86 professions
- Generate the $86 \times 219$ matrix of translations and back-translations across the 109 languages being offered by the Google translate API
( $219 = 109 \times 2 +1) $


In [ ]:
import sys
import os
from tqdm import tqdm_notebook
from google.colab import drive
import importlib
importlib.reload(sys)
#######################
drive.flush_and_unmount()
drive.mount('/gdrive', force_remount=True)
# Enter your own proj_dir here
proj_dir='/gdrive/My Drive/Colab Notebooks/Backtranslation_contronym'
os.chdir(proj_dir)
#############################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
# Cleanliness of displays
np.set_printoptions(suppress=True)
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
%precision 3

Mounted at /gdrive


'%.3f'

# Dataset curation:

- Step-1: Raw word list from the following 3 sources										

1:	https://learnersdictionary.com/3000-words/topic/jobs-professions/1																								
2:	https://gist.githubusercontent.com/mbejda/f08bd3348afad3f06ac1/raw/9d5fcffab6ca778433179547d2daaca6c6ab34f6/professions.txt																								
3:	https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1751-9020.2010.00294.x																								
- Step-2: Singularize and drop-duplicates:

```
df_professions=pd.read_csv('professions_list.csv')
from textblob import Word
df_professions['word_processed']=df_professions.loc[:,'Word'].apply(lambda w: Word(w).singularize()).str.lower()
df_unique=df_professions.drop_duplicates(subset=['word_processed'], keep=False).reset_index().iloc[:,1:]
df_unique.sort_values('word_processed', ascending=True,inplace=True)
np.savetxt('profession_list.txt',df_unique.word_processed.values,fmt='%s')

```

- Step-3: Correct grammatical mistakes using lmproof

```
professions_87=list(pd.read_csv('profession_list_87.txt',header=None)[0].values)
professions_87[0:3]
# ['advocate', 'agriculturist', 'aircraft pilot']
list_sentence=[]
# !pip install lmproof
import lmproof
proof_reader = lmproof.load("en")
for profession in tqdm_notebook(professions_87):
  raw_sentence=f'She is a {profession}.'
  corrected_sentence = proof_reader.proofread(raw_sentence)
  list_sentence.append(corrected_sentence)
```
- Step-4: Hand-correct a few remaining weird grammatical errors

In [ ]:
list_sentence_she=[ 'She is an advocate.',
 'She is an agriculturist.',
 'She is an aircraft pilot.',
 'She is an artist.',
 'She is an astronomer.',
 'She is an attorney.',
 'She is an audiologist.',
 'She is a banker.',
 'She is a barber.',
 'She is a bartender.',
 'She is a biologist.',
 'She is a bookkeeper.',
 'She is a botanist.',
 'She is a builder.',
 'She is a businessperson.',
 'She is a butcher.',
 'She is a carpenter.',
 'She is a cashier.',
 'She is a chef.',
 'She is a chemist.',
 'She is a clergy advisor.',
 'She is a clinical lab technician.',
 'She is a coach.',
 'She is a developer.',
 'She is a dietitian.',
 'She is an ecologist.',
 'She is an editor.',
 'She is an electrician.',
 'She is an EMT specialist.',
 'She is a farmer.',
 'She is a filmmaker.',
 'She is a firefighter.',
 'She is a fisherman.',
 'She is a flight attendant.',
 'She is a geneticist.',
 'She is a geologist.',
 'She is a graphic designer.',
 'She is a human resource specialist.',
 'She is an immunologist.',
 'She is an interpreters.',
 'She is a jeweller.',
 'She is a lawyer.',
 'She is a librarian.',
 'She is a licensed practical nurse.',
 'She is a mechanic.',
 'She is a medical laboratory scientist.',
 'She is a meteorologist.',
 'She is a midwife.',
 'She is a military officer.',
 'She is a nutritionist.',
 'She is an oceanographer.',
 'She is an osteopath.',
 'She is a painter.',
 'She is a paramedic.',
 'She is a pathologist.',
 'She is a pharmacologist.',
 'She is a philosopher.',
 'She is a photographer.',
 'She is a physician assistant.',
 'She is a physicist.',
 'She is a pilot.',
 'She is a plumber.',
 'She is a politician.',
 'She is a receptionist.',
 'She is a registered nurse.',
 'She is a salesman.',
 'She is a salesperson.',
 'She is a scientist.',
 'She is a sea captain.',
 'She is a secretary.',
 'She is a singer.',
 'She is a solicitor.',
 'She is a speech pathologist.',
 'She is a speech-language pathologist.',
 'She is a statistician.',
 'She is a surveyor.',
 'She is a therapist.',
 'She is an undertaker.',
 'She is an urban planner.',
 'She is a videographer.',
 'She is a virologist.',
 'She is a visual artist.',
 'She is a waiter.',
 'She is a web designer.',
 'She is a web developer.',
 'She is a zoologist.']

# Install the google-translate PyPi package and run the example that did inspire this work:
Source: https://www.themarshallproject.org/2020/07/08/she-said-her-husband-hit-her-she-lost-custody-of-their-kids


In [ ]:
!pip install --quiet google_trans_new
from google_trans_new import google_translator 
translator = google_translator()  
##############################################
example_='She Said Her Husband Hit Her. She Lost Custody of Their Kids'
##
translated_en_hi_example = translator.translate(example_, lang_src='en',lang_tgt='hi')  
translated_hi_en_example = translator.translate(translated_en_hi_example, lang_src='hi',lang_tgt='en') 
print(f'{example_} \n {translated_en_hi_example} \n {translated_hi_en_example}')

# Load the list of all the languages that Google translate provides for.
Source: https://cloud.google.com/translate/docs/languages

In [ ]:
df_goog_lang=pd.read_csv('google_languages.csv')
df_goog_lang.head()

,Language,ISO-639-1 Code
0,Afrikaans,af
1,Albanian,sq
2,Amharic,am
3,Arabic,ar
4,Armenian,hy


In [ ]:
list_lang=df_goog_lang.iloc[:,1].values
list_lang

array(['af', 'sq', 'am', 'ar', 'hy', 'az', 'eu', 'be', 'bn', 'bs', 'bg',
       'ca', 'ceb', 'zh-CN', 'zh-TW', 'co', 'hr', 'cs', 'da', 'nl', 'en',
       'eo', 'et', 'fi', 'fr', 'fy', 'gl', 'ka', 'de', 'el', 'gu', 'ht',
       'ha', 'haw', 'he', 'hi', 'hmn', 'hu', 'is', 'ig', 'id', 'ga', 'it',
       'ja', 'jv', 'kn', 'kk', 'km', 'rw', 'ko', 'ku', 'ky', 'lo', 'la',
       'lv', 'lt', 'lb', 'mk', 'mg', 'ms', 'ml', 'mt', 'mi', 'mr', 'mn',
       'my', 'ne', 'no', 'ny', 'or', 'ps', 'fa', 'pl', 'pt', 'pa', 'ro',
       'ru', 'sm', 'gd', 'sr', 'st', 'sn', 'sd', 'si', 'sk', 'sl', 'so',
       'es', 'su', 'sw', 'sv', 'tl', 'tg', 'ta', 'tt', 'te', 'th', 'tr',
       'tk', 'uk', 'ur', 'ug', 'uz', 'vi', 'cy', 'xh', 'yi', 'yo', 'zu'],
      dtype=object)

In [ ]:
# Initialize the translation data-frame
df_translate=pd.DataFrame(data=list_sentence_she,columns=['original'])
df_translate.head()

,original
0,She is an advocate.
1,She is an agriculturist.
2,She is an aircraft pilot.
3,She is an artist.
4,She is an astronomer.


# Function to perform back-translation:

In [ ]:
def backtranslate_lang(lang,list_input=list_sentence_she):
  translated_en_lang=[]
  # Note: Passing the entire sentence list yields _weird_ formatting errors.
  # The only method that seems to work is literal for-looping over the individual sentences
  
  # Step-1: Foward translation
  for sentence_i in list_input:
    translated_en_lang_i = translator.translate(sentence_i, lang_src='en',lang_tgt=lang)  
    translated_en_lang.append(translated_en_lang_i)
    
  # Step-2: Back-translation
  translated_lang_en=[]
  for sentence_i in translated_en_lang:
    translated_lang_en_i = translator.translate(sentence_i, lang_src=lang,lang_tgt='en')  
    translated_lang_en.append(translated_lang_en_i)
  lang_name=df_goog_lang.loc[df_goog_lang.iloc[:,1]==lang].Language.values[0]
  df_lang=pd.DataFrame(data=np.stack([translated_en_lang,translated_lang_en]).T,
                        columns=[f'En2{lang_name}',f'{lang_name}2En'])
  return df_lang

# For-loop over all 109 languages:

## Using Google translate API
(Warning: This is severely rate limited and you may encounter this: *google_new_trans Error: 429 (Too Many Requests)  from TTS API. Probable cause: Unknown*)

Quick-fix: You might have to use the 'Hit factory reset runtime' should you run into API-access-rate errors
```
for lang in tqdm_notebook(list_lang):
  try:
    df_lang=backtranslate_lang(lang,list_input=list_sentence_she)
    df_translate=pd.concat([df_translate, df_lang], axis=1)
    print(df_translate.shape)
    df_translate.to_csv('df_translation_all.csv',index=False)
  except:
    print(f'{lang} failed!')

df_translate.to_csv('df_bt_prof.csv',index=False)

```

In [ ]:
df_bt_professions=pd.read_csv('df_bt_prof.csv')
df_bt_professions.shape
# 1+(109 x 2)= 219 columns

(86, 219)

In [ ]:
df_bt_professions

,Original,En2Hindi,Hindi2En,En2Urdu,Urdu2En,En2Afrikaans,Afrikaans2En,En2Albanian,Albanian2En,En2Amharic,Amharic2En,En2Arabic,Arabic2En,En2Armenian,Armenian2En,En2Azerbaijani,Azerbaijani2En,En2Basque,Basque2En,En2Belarusian,Belarusian2En,En2Bengali,Bengali2En,En2Bosnian,Bosnian2En,En2Bulgarian,Bulgarian2En,En2Catalan,Catalan2En,En2Cebuano,Cebuano2En,En2Chinese_Simplified,Chinese_Simplified2En,En2Chinese_Traditional,Chinese_Traditional2En,En2Corsican,Corsican2En,En2Croatian,Croatian2En,En2Czech,...,En2Sundanese,Sundanese2En,En2Swahili,Swahili2En,En2Swedish,Swedish2En,En2Tagalog,Tagalog2En,En2Tajik,Tajik2En,En2Tamil,Tamil2En,En2Tatar,Tatar2En,En2Telugu,Telugu2En,En2Thai,Thai2En,En2Turkish,Turkish2En,En2Turkmen,Turkmen2En,En2Ukrainian,Ukrainian2En,En2Uyghur,Uyghur2En,En2Uzbek,Uzbek2En,En2Vietnamese,Vietnamese2En,En2Welsh,Welsh2En,En2Xhosa,Xhosa2En,En2Yiddish,Yiddish2En,En2Yoruba,Yoruba2En,En2Zulu,Zulu2En
0,She is an advocate.,वह एक वकील है।,He is a lawyer.,وہ ایک وکیل ہے۔,He is a lawyer.,Sy is 'n advokaat.,She's a lawyer.,Ajo është avokate.,She is a lawyer.,ተሟጋች ናት ፡፡,She is an advocate.,هي مناصرة.,She is an advocate.,Նա փաստաբան է:,He is a lawyer.,O vəkildir.,He is a lawyer.,Abokatua da.,He is a lawyer.,Яна абаронца.,She is a defender.,তিনি একজন অ্যাডভোকেট।,He is an advocate.,Ona je advokat.,She is a lawyer.,Тя е адвокат.,She is a lawyer.,És una defensora.,She is an advocate.,Siya usa ka manlalaban.,He is a defender.,她是一位拥护者。,She is a champion.,她是一位擁護者。,She is a champion.,Hè un difensore.,He is a defender.,Ona je odvjetnica.,She's a lawyer.,Je obhájkyní.,...,Anjeunna mangrupikeun pangacara.,He is a lawyer.,Yeye ni wakili.,He is a lawyer.,Hon är en advokat.,She's a lawyer.,Siya ay isang tagapagtaguyod.,He is an advocate.,Вай адвокат аст.,She is a lawyer.,அவள் ஒரு வக்கீல்.,She is a lawyer.,Ул яклаучы.,He is a defender.,ఆమె న్యాయవాది.,She is a lawyer.,เธอเป็นผู้สนับสนุน,She is a supporter,O bir avukattır.,"['He is a lawyer.', 'She is a lawyer.']",Aklawçy.,Lawyer.,Вона є адвокатом.,She is a lawyer.,ئۇ بىر ئادۋوكات.,He is a lawyer.,U advokat.,He is a lawyer.,Cô ấy là một người ủng hộ.,She is an advocate.,Mae hi'n eiriolwr.,She is an advocate.,Ungummeli.,He is a lawyer.,זי איז אַן אדוואקאט.,She is a lawyer.,O jẹ alagbawi.,He is an advocate.,Ungummeli.,He is a lawyer.
1,She is an agriculturist.,वह एक कृषक है।,He is a farmer.,وہ ایک زراعت پسند ہے۔,He is an agriculturist.,Sy is 'n landboukundige.,She is an agriculturist.,Ajo është një bujqësore.,It is an agricultural.,እርሷ የግብርና ባለሙያ ናት ፡፡,She is an agronomist.,هي زراعيّة.,It is agricultural.,Նա գյուղատնտես է:,He is an agriculturist.,"O, kənd təsərrüfatı mütəxəssisidir.",He is an agricultural specialist.,Nekazaria da.,He is a farmer.,Яна сельская гаспадарка.,She is farming.,তিনি একজন কৃষিবিদ।,He is an agriculturist.,Ona je poljoprivrednica.,She is a farmer.,Тя е земеделец.,She is a farmer.,És agricultora.,She is a farmer.,Usa siya ka Agrikulturista.,He is an Agriculturalist.,她是一位农业学家。,She is an agriculturist.,她是一位農業學家。,She is an agriculturist.,Hè agricultore.,He is a farmer.,Ona je poljoprivrednica.,She is a farmer.,Je to zemědělkyně.,...,Anjeunna ahli pertanian.,He is an agricultural expert.,Yeye ni mtaalam wa kilimo.,He is an agricultural expert.,Hon är jordbrukare.,She is a farmer.,Siya ay isang magsasaka.,He is a farmer.,Вай кишоварз аст.,She is a farmer.,அவள் ஒரு விவசாயி.,She is a farmer.,Ул агроном.,Ul agronomist.,ఆమె వ్యవసాయదారుడు.,She is a farmer.,เธอเป็นเกษตรกร,She is a farmer,Tarımcıdır.,It is agricultural.,Ol agronom.,He is an agronomist.,Вона аграрій.,She is an agrarian.,ئۇ دېھقانچىلىقچى.,He is a farmer.,U qishloq xo'jaligi mutaxassisi.,He is an agricultural specialist.,Cô ấy là một nhà nông nghiệp.,She is an agriculturalist.,Mae hi'n amaethwr.,She is a farmer.,Uyingcali kwezolimo.,He is an agricultural expert.,זי איז אַ לאַנדווירטשאַפטלעך.,She is an agricultural.,Arabinrin agbe ni.,She is a farmer's sister.,Ungumlimi.,He is a farmer.
2,She is an aircraft pilot.,वह एक एयरक्राफ